In [1]:
%load_ext autoreload
%autoreload 2
from qiskit_metal import draw, Dict, designs, MetalGUI
from qiskit_metal.toolbox_metal import math_and_overrides
from qiskit_metal.qlibrary.core import QComponent
import qiskit_metal as metal
from just_claw import TransmonClaw
# ?metal.qlibrary.core.QComponent.add_qgeometry

design = metal.designs.design_planar.DesignPlanar()
gui = metal.MetalGUI(design)
design.overwrite_enabled = True

# dir(QComponent)
cpw_width = '11.7um'

design._chips.main.size.size_x = '5mm'
design._chips.main.size.size_y = '5mm'
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.tlines.mixed_path import RouteMixed
from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.couplers.cap_n_interdigital_tee import CapNInterdigitalTee
from qiskit_metal.qlibrary.couplers.line_tee import LineTee

from collections import OrderedDict

import numpy as np

xmon1_options = dict(
    connection_pads = dict(
        readout = dict(
            connector_location='90',
            connector_type='0',
            claw_length="150um",
            ground_spacing='4.1um',
            claw_gap='5.1um',
            claw_width='15um',
            claw_cpw_length=0
        ),
    ),
    cross_width='30um',
    cross_length="100um",
    cross_gap='30um',
    orientation='-90',
    pos_x='-1500um',
    pos_y='1200um',
    # aedt_hfss_inductance=inductance,
    # hfss_inductance=inductance, 
    # q3d_inductance=inductance,
    # aedt_q3d_inductance=inductance
)

# Create a new Transmon Cross object with name 'Q1'
# q = TransmonClaw(design, 'Q', options=xmon1_options)

clt1_opts = Dict(prime_width = cpw_width,
                prime_gap = '5.1um',
                second_width = cpw_width,
                second_gap = '5.1um',
                coupling_space = '7.9um',
                coupling_length = '225um',
                open_termination = False,
                orientation = '-90',
                pos_y = '1200um',
                down_length = '50um')

# clt = CoupledLineTee(design, 'clt', clt1_opts)

cpw1_opts = Dict(pin_inputs = Dict(start_pin = Dict(component = 'clt',
                                                    pin = 'second_end'),
                                   end_pin = Dict(component = 'Q',
                                                  pin = 'readout')),
                lead = Dict(start_straight = '50um',
                            # start_jogged_extension = jogsS),
                ),
                fillet = '49.9um',
                total_length = '3885.5930858724796um',#'3893.8166um', # '6776um',
                trace_width = cpw_width,
                meander = Dict(spacing = '100um',
                               asymmetry = '-115um'),
                trace_gap = '5.1um',)
# cpw = RouteMeander(design, 'cpw', options = cpw1_opts)

# gui.rebuild()
# gui.autoscale()
# gui.zoom_on_components(['clt5', 'Q5'])
# gui.screenshot()

In [2]:
Lj_vals = [14.82977874,12.76661344,11.5666809,14.02940303,9.329626635]
predicted_cross_length = [172.2144875,201.49286,223.1790728,182.8509009,274.2789866]
predicted_claw_length = [122.6728936,168.3926037,200.9612748,176.5367286,55.67607171]
cpw_lengths = [4022.333844,3833.908348,3645.492252,4216.087604,4021.494855]
cplr_lengths = [232.5421308,238.0444714,230.2712219,206.6035852,218.3920977]

In [3]:
ground_spacing = 4.1

In [4]:
comps_available = {
    "Sabrina":0,
    "Glinda":1,
    "Saruman":2,
    "VM1":3, #?
    "VM2":4, #?
}

In [5]:
indexes = (0,1)
version = "fig4_v14_q4test"
comp_id = "Sabrina"  # Replace with actual computer name

# Eigenmode Sim (CLT + CPW + Claw)

In [6]:
from qiskit_metal.analyses.quantization import EPRanalysis
import pyEPR as epr
from pyaedt import Hfss
from utils import *
from objects import *
from datetime import datetime
import pprint

In [7]:
eig_q12bus = EPRanalysis(design, "hfss")

In [8]:
%matplotlib inline
eig_q12bus = EPRanalysis(design, "hfss")
hfss = eig_q12bus.sim.renderer

for i, (cross_length, claw_length, Lj, cpw_len, cplr_len) in enumerate(zip(predicted_cross_length, predicted_claw_length, Lj_vals, cpw_lengths, cplr_lengths)):
    if i not in indexes:
        continue
    # print(i)
    xmon1_options["connection_pads"]["readout"]["claw_length"] = f"{claw_length}um"
    xmon1_options["connection_pads"]["readout"]["ground_spacing"] = f"{ground_spacing}um"
    xmon1_options["aedt_hfss_inductance"]=Lj,
    xmon1_options["hfss_inductance"]=Lj, 
    xmon1_options["q3d_inductance"]=Lj,
    xmon1_options["aedt_q3d_inductance"]=Lj

    cpw1_opts["total_length"] = f"{cpw_len}um"
    clt1_opts["coupling_length"] = f"{int(round(cplr_len, 0))}um"
    
    v2_opts = Dict(
        claw_opts = xmon1_options,
        cplr_opts = clt1_opts,
        cpw_opts = cpw1_opts
    )
    # pprint.pprint(v2_opts)
    now = datetime.now()
    #print(xmon1_options)
    #print(cpw1_opts)
    #print(clt1_opts)
    print()
    CLT_epr_sweep(design = design, sweep_opts=v2_opts, filename = f"cavity_claw_num{i}_{comp_id}_v{version}")


INFO 02:01PM [connect_project]: Connecting to Ansys Desktop API...
INFO 02:01PM [load_ansys_project]: 	Opened Ansys App
INFO 02:01PM [load_ansys_project]: 	Opened Ansys Desktop v2021.1.0
INFO 02:01PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    D:/lfl/Documents/Ansoft/
	Project:   fig4_sims



Starting the Simulation


INFO 02:01PM [connect_design]: 	Opened active design
	Design:    LOMv2.0_q3d11 [Solution type: Q3D]
INFO 02:01PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 02:01PM [connect]: 	Connected to project "fig4_sims" and design "LOMv2.0_q3d11" 😀 

INFO 02:01PM [connect_design]: 	Opened active design
	Design:    CavitySweep20 [Solution type: Eigenmode]
WARNING 02:01PM [connect_setup]: 	No design setup detected.
WARNING 02:01PM [connect_setup]: 	Creating eigenmode default setup.
INFO 02:01PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


INFO 02:01PM [connect_design]: 	Opened active design
	Design:    CavitySweep_hfss [Solution type: Eigenmode]


creaing porst


INFO 02:01PM [connect_project]: Connecting to Ansys Desktop API...
INFO 02:01PM [load_ansys_project]: 	Opened Ansys App
INFO 02:01PM [load_ansys_project]: 	Opened Ansys Desktop v2021.1.0
INFO 02:01PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    D:/lfl/Documents/Ansoft/
	Project:   fig4_sims


Sim rendered into HFSS!
{'mesh1': {'objects': ['prime_cpw_cplr', 'second_cpw_cplr', 'trace_cpw', 'readout_connector_arm_claw'], 'MaxLength': '7um'}}

Starting the Simulation


INFO 02:01PM [connect_design]: 	Opened active design
	Design:    CavitySweep_hfss [Solution type: Eigenmode]
INFO 02:01PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 02:01PM [connect]: 	Connected to project "fig4_sims" and design "CavitySweep_hfss" 😀 

INFO 02:01PM [connect_design]: 	Opened active design
	Design:    CavitySweep21 [Solution type: Eigenmode]
WARNING 02:01PM [connect_setup]: 	No design setup detected.
WARNING 02:01PM [connect_setup]: 	Creating eigenmode default setup.
INFO 02:01PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


INFO 02:01PM [connect_design]: 	Opened active design
	Design:    CavitySweep_hfss [Solution type: Eigenmode]


creaing porst
Sim rendered into HFSS!
{'mesh1': {'objects': ['prime_cpw_cplr', 'second_cpw_cplr', 'trace_cpw', 'readout_connector_arm_claw'], 'MaxLength': '7um'}}


# LOM Analysis

In [9]:
# xmon = TransmonCross(design = design, options = xmon1_options, name = "xmon")

In [10]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [11]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [12]:
c1.sim.setup.reuse_selected_design = False
c1.sim.setup.reuse_setup = False

# example: update single setting
c1.sim.setup.max_passes = 30
c1.sim.setup.min_converged_passes = 1
c1.sim.setup.percent_error = 0.1
c1.sim.setup.name = 'sweep_setup'

In [13]:
def do_LOM_qubitonly(cross_length, claw_length, Lj, i):
    qname = 'xmon'
    cname = 'readout'

    # design.components[qname].options['connection_pads']['readout']['claw_length'] = f'{claw_length}um'
    # design.components[qname].options['cross_length'] = f'{cross_length}um'
    # design.components[qname].options['hfss_inductance'] = Lj
    # design.components[qname].options['aedt_hfss_inductance'] = Lj
    # design.components[qname].options['q3d_inductance'] = Lj
    # design.components[qname].options['aedt_q3d_inductance'] = Lj
    xmon1_options["cross_length"] = f"{cross_length}um"
    xmon1_options["connection_pads"]["readout"]["claw_length"] = f"{claw_length}um"
    xmon1_options["connection_pads"]["readout"]["ground_spacing"] = f"{ground_spacing}um"
    xmon1_options["aedt_hfss_inductance"]=Lj,
    xmon1_options["hfss_inductance"]=Lj, 
    xmon1_options["q3d_inductance"]=Lj,
    xmon1_options["aedt_q3d_inductance"]=Lj
    q = TransmonCross(design, qname, options=xmon1_options)
    design.rebuild()
    selection = [qname]
    open_pins = [(qname, cname)]
    print(q.options)
    c1.sim.renderer.clean_active_design()
    c1.sim.run(name = 'LOMv2.0', components=selection,
               open_terminations=open_pins)
    cap_df = c1.sim.capacitance_matrix

    print(f'For Qubit_{i}, the cap matrix is...')
    print(cap_df)

    data = {
        "design_options": design.components[qname].options,
        "data": {
            "cross_to_ground": 0 if 'ground_main_plane' not in cap_df.loc[f'cross_{qname}'] else cap_df.loc[f'cross_{qname}']['ground_main_plane'],
            "claw_to_ground": 0 if 'ground_main_plane' not in cap_df.loc[f'{cname}_connector_arm_{qname}'] else cap_df.loc[f'{cname}_connector_arm_{qname}']['ground_main_plane'],
            "cross_to_claw": cap_df.loc[f'cross_{qname}'][f'{cname}_connector_arm_{qname}'],
            "cross_to_cross": cap_df.loc[f'cross_{qname}'][f'cross_{qname}'],
            "claw_to_claw": cap_df.loc[f'{cname}_connector_arm_{qname}'][f'{cname}_connector_arm_{qname}'],
            "ground_to_ground": 0 if 'ground_main_plane' not in cap_df.loc[f'cross_{qname}'] else cap_df.loc['ground_main_plane']['ground_main_plane']
        },
        "sim_info": {
            "setup": c1.sim.setup,
            "renderer_options": c1.sim.renderer.options
        }
    }
    save_simulation_data_to_json(data, filename = f"qubitonly_num{i}_{comp_id}_v{version}")


In [14]:
all_cap_matricies = []

for i, (cross_length, claw_length, Lj) in enumerate(zip(predicted_cross_length, predicted_claw_length, Lj_vals)):
    # print(claw_length)
    if i not in indexes:
        continue
    do_LOM_qubitonly(cross_length, claw_length, Lj, i)

INFO 02:01PM [connect_project]: Connecting to Ansys Desktop API...
INFO 02:01PM [load_ansys_project]: 	Opened Ansys App
INFO 02:01PM [load_ansys_project]: 	Opened Ansys Desktop v2021.1.0
INFO 02:01PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    D:/lfl/Documents/Ansoft/
	Project:   fig4_sims


{'pos_x': '-1500um', 'pos_y': '1200um', 'orientation': '-90', 'chip': 'main', 'layer': '1', 'connection_pads': {'readout': {'connector_type': '0', 'claw_length': '122.6728936um', 'ground_spacing': '4.1um', 'claw_width': '15um', 'claw_gap': '5.1um', 'claw_cpw_length': 0, 'claw_cpw_width': '10um', 'connector_location': '90'}}, 'cross_width': '30um', 'cross_length': '172.2144875um', 'cross_gap': '30um', 'hfss_inductance': (14.82977874,), 'hfss_capacitance': 0, 'hfss_resistance': 0, 'hfss_mesh_kw_jj': 7e-06, 'q3d_inductance': (14.82977874,), 'q3d_capacitance': 0, 'q3d_resistance': 0, 'q3d_mesh_kw_jj': 7e-06, 'gds_cell_name': 'my_other_junction', 'aedt_q3d_inductance': 14.82977874, 'aedt_q3d_capacitance': 0, 'aedt_hfss_inductance': (14.82977874,), 'aedt_hfss_capacitance': 0}


INFO 02:01PM [connect_design]: 	Opened active design
	Design:    CavitySweep_hfss [Solution type: Eigenmode]
INFO 02:01PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 02:01PM [connect]: 	Connected to project "fig4_sims" and design "CavitySweep_hfss" 😀 

INFO 02:02PM [connect_design]: 	Opened active design
	Design:    LOMv2.0_q3d12 [Solution type: Q3D]
WARNING 02:02PM [connect_setup]: 	No design setup detected.
WARNING 02:02PM [connect_setup]: 	Creating Q3D default setup.
INFO 02:02PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)


<COMObject GetModule>


INFO 02:02PM [get_setup]: 	Opened setup `sweep_setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 02:02PM [analyze]: Analyzing setup sweep_setup
INFO 02:02PM [__del__]: Disconnected from Ansys HFSS


com_error: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147024349), None)